In [6]:
import numpy as np
from sklearn.cluster import SpectralClustering
import skfuzzy as fuzz
import pandas as pd


# Set random seed for reproducibility
np.random.seed(42)

# Simulation parameters
num_providers = 3
num_regions = 3
min_dcs_per_provider = 2
max_dcs_per_provider = 5
num_vms_per_dc = 8
vm_processing_capability = 1500  # in MIPS
vm_num_cpus = 2
vm_ram = 4  # in Gb
vm_storage_capacity = 8  # in Gb
num_data = 200  # Number of data
data_size_range = (300, 1000)  # Data size range in Mb
task_size_range = (200, 1000)  # Task size range in MI
Re = 0.7  # Provider revenues per task execution ($)
C_penalty = 0.0025  # Penalty per violation ($)


inter_region_bw_capacity = 500  # in Mb/s
inter_region_bw_delay = 150  # in ms

intra_region_bw_capacity = 1000  # in Mb/s
intra_region_bw_delay = 50  # in ms

intra_dc_bw_capacity = 8000  # in Mb/s
intra_dc_bw_delay = 10  # in ms

# Pricing information
provider_pricing = {
    'Provider 1': {
        'US': [0.020, 0.006, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'EU': [0.025, 0.006, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'AS': [0.027, 0.0066, 0.001, 0.0015, 0.002, 0.004, 0.008],
    },
    'Provider 2': {
        'US': [0.020, 0.0096, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'EU': [0.018, 0.0096, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'AS': [0.020, 0.0096, 0.001, 0.0015, 0.002, 0.004, 0.008],
    },
    'Provider 3': {
        'US': [0.0095, 0.00120, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'EU': [0.0090, 0.0096, 0.001, 0.0015, 0.002, 0.004, 0.008],
        'AS': [0.0080, 0.0090, 0.001, 0.0015, 0.002, 0.004, 0.008],
    },
}

avg_response_times = {}

# Simulate data for each provider and region
# Vary the number of data centers

for num_dcs in num_dc_list:
    provider_data = []  # Reset provider data for each iteration
    for provider_id in range(1, num_providers + 1):
        for region in ['US', 'EU', 'AS']:
            for dc_id in range(1, num_dcs + 1):                
                for data_id in range(1, num_data + 1):
                    data_size = np.random.uniform(data_size_range[0], data_size_range[1])
                    task_size = np.random.uniform(task_size_range[0], task_size_range[1])
                    
                    base_response_time = 180
                    response_time_variation = np.random.normal(loc=0, scale=10)
                    response_time = max(0, base_response_time + response_time_variation)
                    
                    # Include bandwidth and delay information in the data
                    vm_data = {
                        'provider_id': provider_id,
                        'region': region,
                        'dc_id': dc_id,
                        'num_vms': num_vms_per_dc,
                        'vm_processing_capability': vm_processing_capability,
                        'vm_num_cpus': vm_num_cpus,
                        'vm_ram': vm_ram,
                        'vm_storage_capacity': vm_storage_capacity,
                        'cpu_price': provider_pricing[f'Provider {provider_id}'][region][0],
                        'storage_price': provider_pricing[f'Provider {provider_id}'][region][1],
                        'intra_dc_bw_price': provider_pricing[f'Provider {provider_id}'][region][2],
                        'inter_region_bw_price': provider_pricing[f'Provider {provider_id}'][region][3],
                        'intra_region_bw_price': provider_pricing[f'Provider {provider_id}'][region][4],
                        'response_time_slo': response_time,
                        'availability_slo': 0.95,
                        'task_count': np.random.choice([1000, 2000, 3000, 5000, 7000, 10000]),
                        'data_size': data_size,
                        'task_size': task_size,
                        'inter_region_bw_capacity': inter_region_bw_capacity,
                        'inter_region_bw_delay': inter_region_bw_delay,
                        'intra_region_bw_capacity': intra_region_bw_capacity,
                        'intra_region_bw_delay': intra_region_bw_delay,
                        'intra_dc_bw_capacity': intra_dc_bw_capacity,
                        'intra_dc_bw_delay': intra_dc_bw_delay,
                    }
                    provider_data.append(vm_data)

    # 'provider_data' includes task-related features
features = [
    'cpu_price', 'storage_price', 'intra_dc_bw_price', 'inter_region_bw_price',
    'total_cost', 'task_count', 'data_size', 'task_size',
    'response_time_slo', 'availability_slo',
    'inter_region_bw_capacity', 'inter_region_bw_delay',
    'intra_region_bw_capacity', 'intra_region_bw_delay',
    'intra_dc_bw_capacity', 'intra_dc_bw_delay', 'provider_id',
]
data = np.array([[vm.get(feature, 0) for feature in features] for vm in provider_data])

# Create a DataFrame
df = pd.DataFrame(data, columns=features)

df.to_csv('simulated_data.csv')